In [17]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import os
import utils

from pprint import pprint

In [18]:
# Paths
clusters_dir = '../../../all_clusters'
data_dir = 'data'
figures_dir = 'figures'

# Create figures directory if none exists
if not os.path.exists(figures_dir):
        os.makedirs(figures_dir)

# Load all data
tloops_raw = utils.load(f'{data_dir}/tloops_raw.pickle')
tloops_filtered = utils.load(f'{data_dir}/tloops_filtered.pickle')

# Tetraloops
> The final tetraloop database is composed by 21,993 RNA fragments: 17,709 from X-ray, 3057 from NMR and 1227 from cryo-EM structures. The distributions of resolutions are shown in Fig. S1. [Bottaro et al.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5529312/)

According to Table SI1, there should be a total of 19383 tetraloops. I don't know where/what the remaining 2610 RNA fragments (from the total of 21993) are. The total number of "effective" tetraloops (members whose distance is above 0.07 eRMSD) is 16979.

In [19]:
def tloop_stats(tloop_data, label:str ='') -> None:
    seqs = [i.res_seq for i in tloop_data]
    unique_seqs = set(seqs)
    
    print(
        f'{label}:\n'
        f'- Number: {len(tloop_data)}\n'
        f'- Unique sequences: {len(unique_seqs)}\n'
    )
    
    # Sequence histogram
    seqs_hist = {seq:seqs.count(seq) for seq in unique_seqs}
    plot_bar(seqs_hist, f'{label} sequence frequencies', sort_by_val=True)
    
    # Sequence by cluster
    for c in range(1,45):
        clust_seqs = [t.res_seq for t in tloop_data if t.clust_id == c]
        clust_seqs_hist = {seq:clust_seqs.count(seq) for seq in set(clust_seqs)}
        plot_bar(clust_seqs_hist, f'{label} cluster {c} sequence frequencies', sort_by_val=True)
    
    # Cluster histogram
    clust_size_hist = {c:len([t for t in tloop_data if t.clust_id == c]) for c in range(1,45)}
    plot_bar(clust_size_hist, f'{label} cluster frequencies')


def plot_bar(data, title:str='', sort_by_val:bool=False) -> None:
    data = {k:v for k,v in sorted(data.items(), key=lambda x:x[1], reverse=True)} if sort_by_val else data
    fig, ax = plt.subplots(figsize=(len(data)*0.3,3))
    bar_plot = ax.bar(range(len(data)), data.values(), align='edge', width=0.5)
    ax.bar_label(bar_plot, rotation='vertical', padding=5)
    ax.set_xticks(range(len(data)), data.keys(), rotation='vertical', horizontalalignment='left')
    
    # Cosmetic adjustments
    ax.set_title(title, y=1.1)
    ax.margins(x=0.5/len(data), tight=True)
    ax.spines[['right', 'top']].set_visible(False)

    filename = title.lower().replace(' ','_')
    plt.savefig(f'{figures_dir}/{filename}.pdf', bbox_inches='tight')
    plt.close()

In [20]:
tloop_stats(tloops_raw, 'Raw tetraloops')
tloop_stats(tloops_raw, 'Filtered tetraloops')

Raw tetraloops:
- Number: 19383
- Unique sequences: 292

Filtered tetraloops:
- Number: 19383
- Unique sequences: 292



In [ ]:
# Tetraloop fragment dataset comparisons

print(f'Unique tetraloops: {len(tloop_seqs)}')
tloop_clust_nums = {clust: len([i for i in tloop_seqs if i.clust_id == clust]) for clust in range(1,45)}
print(tloop_clust_nums)

print(f'Unique aligned tetraloops: {len([i for i in all_fragments_8 if i.clust_id != 0])}')
aligned_tloop_clust_nums = {clust: len([i for i in all_fragments_8 if i.clust_id == clust]) for clust in range(1,45)}
print(aligned_tloop_clust_nums)

clust_num_diff = {i:tloop_clust_nums[i]-aligned_tloop_clust_nums[i] for i in range(1,45) if tloop_clust_nums[i] != aligned_tloop_clust_nums[i]}
print(f'Number of missing tetraloops: {clust_num_diff}')

# Fragments

## Unique fragments
The number of unique RNA fragments is $4^{8} = 65536$. The theoretical maximum number of fragments that can be generated from this dataset is 495279. The number of unique generated fragments is 34431, which represents approximately 53% of all possible unique fragments. 

Expected number of tries to obtain $x$ unique sequences:
$$\sum\limits_{i=0}^x \frac{n}{n-i}$$
where $n$ = total number of unique sequences

The expected number of tries needed to obtain *all* unique fragments is 764646. The expected number of tries needed to obtain 34431 fragments is 48839.

The dataset is most likely too small to account for *all* possible residue permutations, but should be large enough to account for more than 53% of all residues. For reference, a dataset of size 451750 is expected to return 99.9% of all unique fragments.

- Are certain residues/motifs overrepresented in the dataset?
- Will increasing the dataset to include more PDB structures result in more unique fragments?
- Is the sequence database too redundant? Should sequence similarity *between* different PDB structures also be filtered out of the reference database?
- Are the sequences palindromic or otherwise symmetrical?

In [ ]:
# Fragment length 8

pos_unique_frags = 4**8
print(f'Possible unique fragments: {pos_unique_frags}')

pos_gen_frags = sum([len(i.res_seq)-7-8*(len(i.chain_indices)-1) for i in aligned_chains])
print(f'Generated fragments (theoretical): {pos_gen_frags}')
gen_frags = [i.res_seq for i in fragments_8]
gen_unique_frags = len(set(gen_frags))
print(f'Number of unique fragments: {gen_unique_frags}')
print(f'% of expected unique fragments: {round(gen_unique_frags/pos_unique_frags, 2)*100}%')
frags_dict = {seq:gen_frags.count(seq) for seq in set(gen_frags)}
frags_dict = {k: v for k, v in sorted(frags_dict.items(), key=lambda item: item[1], reverse=True)}
print(f'Fragment counts: {frags_dict}')

expected_tries_all = sum([pos_unique_frags/(pos_unique_frags-i) for i in range(pos_unique_frags)])
print(f'Expected # tries to obtain all unique fragments: {round(expected_tries_all)}')
expected_tries_34431 = sum([pos_unique_frags/(pos_unique_frags-i) for i in range(gen_unique_frags)])
print(f'Expected # tries to obtain 34431 of all unique fragments: {round(expected_tries_34431)}')
expected_tries_999 = sum([pos_unique_frags/(pos_unique_frags-i) for i in range(int(pos_unique_frags*0.999))])
print(f'Expected # tries to obtain 99.9% unique fragments: {round(expected_tries_999)}')

## Per-position residue frequency
G seems to be slightly enriched in all positions.

A small difference in the number of tetraloops is observed between the tloop_seqs.pickle and all_fragment_8.pickle datasets due to:
1. PDB chain redundancy removal, which probably unintentionally removes some tetraloop sites
2. Clusters 29 and 30 have an overlapping tetraloop, so either 29 or 30 is overwritten at that position

In [ ]:
# Residue frequency functions

# Get residue frequencies per position
def get_res_freqs(sequences, res_names: list[str] = ['A','U','C','G']):
    res_freqs = {pos: {res: 0 for res in res_names} for pos in range(len(sequences[0]))}
    for seq in sequences:
        for pos, res in enumerate(seq):
            res_freqs[pos][res] += 1
    # Normalize frequencies
    res_freqs = {pos:{res: count/sum(res_counts.values()) for res, count in res_counts.items()} for pos, res_counts in res_freqs.items()}
    return res_freqs


# Convert residue frequency dictionary to a numpy array
def res_freqs_dict_to_array(res_freqs_dict, res_names: list[str] = ['A','U','C','G']):
    res_array= np.zeros(shape=(len(res_names), len(res_freqs_dict)))
    for idx, res in enumerate(res_names):
        res_array[idx] = [res_freqs_dict[pos][res] for pos in range(len(res_freqs_dict))]
    return res_array


# Plot a stacked bar graph
def plot_stacked_bars(x, array, title, labels):#, colors):
    plt.figure(figsize=(5,3))
    plt.bar(x, array[0], label=labels[0])
    for i in range(1, len(array)):
        plt.bar(x, array[i], bottom=np.sum(array[:i], axis=0), label=labels[i])
    plt.title(title)
    plt.legend(loc='upper right', bbox_to_anchor=(1.2,1.03))
    plt.show()


# Combine residue frequency calculation and plotting
def plot_res_freqs(seqs, title: str = '', labels: list[str] = ['A','U','C','G'], print_freqs = False):
    res_freqs = get_res_freqs(seqs)
    if print_freqs:
        pprint({pos:{res:round(count, 3) for res, count in val.items()} for pos, val in res_freqs.items()})
    plot_stacked_bars(range(1, len(seqs[0])+1), res_freqs_dict_to_array(res_freqs), title, labels)

In [ ]:
# Plot residue frequency figures

# Cluster groups, as named in Bottaro et al. Table SI1
gnra = [1]
gnra_like = [1, 3, 6, 9, 25, 26, 36, 40]
uncg = [2]
uncg_like = [2, 5, 37, 44]
u_turn = [4]

plot_res_freqs([i.res_seq for i in all_fragments_8 if i.clust_id in gnra_like], 'GNRA fragment 8')
plot_res_freqs([i.res_seq for i in all_fragments_10 if i.clust_id in gnra_like], 'GNRA fragment 10')
plot_res_freqs([i.res_seq for i in all_fragments_12 if i.clust_id in gnra_like], 'GNRA fragment 12')
plot_res_freqs([i.res_seq for i in all_fragments_14 if i.clust_id in gnra_like], 'GNRA fragment 14')

plot_res_freqs([i.res_seq for i in all_fragments_8 if i.clust_id == 0], 'Cluster 0', print_freqs=True)